In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Problem Statement

## Background

"The Home Mortgage Disclosure Act (HMDA) requires many financial institutions to maintain, report, and publicly disclose loan-level information about mortgages. These data help show whether lenders are serving the housing needs of their communities; they give public officials information that helps them make decisions and policies; and they shed light on lending patterns that could be discriminatory. The public data are modified to protect applicant and borrower privacy.

HMDA was originally enacted by Congress in 1975 and is implemented by Regulation C." - [source](https://www.consumerfinance.gov/data-research/hmda/)

## Resources

[Download the Dataset](https://ffiec.cfpb.gov/data-browser/data/2021?category=states&items=OR)

[2021 HMDA Documentation](https://ffiec.cfpb.gov/documentation/2021/)

[2021 data-feild specification](https://ffiec.cfpb.gov/documentation/2021/lar-data-fields/)

[2021 HMDA Guide](https://www.ffiec.gov/hmda/pdf/2021Guide.pdf)

# Data Loading

In [ ]:
path = '/content/drive/MyDrive/CS510_CulturalCompetenceInComputing/Final submission/'

In [ ]:
df_2021 = pd.read_csv(path + '2021_state_OR.csv')
df_2020 = pd.read_csv(path + '2020_state_OR.csv')
df_2019 = pd.read_csv(path + '2019_state_OR.csv')
df_2018 = pd.read_csv(path + '2018_state_OR.csv')

<ipython-input-4-bff08d619a5e>:1: DtypeWarning: Columns (5,22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2021 = pd.read_csv(path + '2021_state_OR.csv')
<ipython-input-4-bff08d619a5e>:2: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020 = pd.read_csv(path + '2020_state_OR.csv')
<ipython-input-4-bff08d619a5e>:3: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2019 = pd.read_csv(path + '2019_state_OR.csv')
<ipython-input-4-bff08d619a5e>:4: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2018 = pd.read_csv(path + '2018_state_OR.csv')


In [ ]:
print(f"df_2021 = {len(df_2021)}")
print(f"df_2020 = {len(df_2020)}")
print(f"df_2019 = {len(df_2019)}")
print(f"df_2018 = {len(df_2018)}")
print(f"{len(df_2018) + len(df_2019) + len(df_2020) + len(df_2021)} total")

df_2021 = 391601
df_2020 = 400029
df_2019 = 265274
df_2018 = 229722
1286626 total


In [ ]:
for i, col in enumerate(df_2021.columns):
    
    if(df_2020.columns[i] != col):
        print(f"df_2021[{i:02.0f}] = {col}")
        print(f"df_2020[{i:02.0f}] = {df_2020.columns[i]}")
    if(df_2019.columns[i] != col):
        print(f"df_2021[{i:02.0f}] = {col}")
        print(f"df_2019[{i:02.0f}] = {df_2019.columns[i]}")
    if(df_2018.columns[i] != col):
        print(f"df_2021[{i:02.0f}] = {col}")
        print(f"df_2018[{i:02.0f}] = {df_2018.columns[i]}")

In [ ]:
frames = [df_2021, df_2020, df_2019, df_2018]
#frames = [df_2021, df_2020, df_2019]
#frames = [df_2021]
df_read = pd.concat(frames).reset_index(drop=True)

In [ ]:
df = df_read[[
        # Label Field
        'action_taken',

        # Input Fields
        'loan_type',
        'loan_purpose',
        'business_or_commercial_purpose',
        'loan_to_value_ratio',
        'interest_rate',
        'hoepa_status',
        'loan_term',
        'property_value',
        'construction_method',
        'occupancy_type',
        'income',
        'debt_to_income_ratio',
        'submission_of_application',
        'aus-1',

        # Audit Feilds
        'derived_sex',
        'derived_race',
        'derived_ethnicity'
       ]]

In [ ]:
df = df.sample(frac=1)

# Data Filtering

In [ ]:
df['action_taken'] = df['action_taken'].replace(1, 1)
df['action_taken'] = df['action_taken'].replace(2, 1)
df['action_taken'] = df['action_taken'].replace(3, 0)

In [ ]:
df = df.drop(df[df.action_taken > 3].index).reset_index(drop=True)

df = df.drop(df[df.derived_race == 'Free Form Text Only'].index).reset_index(drop=True)
df = df.drop(df[df.derived_race == 'Race Not Available'].index).reset_index(drop=True)

df = df.drop(df[df.derived_ethnicity == 'Free Form Text Only'].index).reset_index(drop=True)
df = df.drop(df[df.derived_ethnicity == 'Ethnicity Not Available'].index).reset_index(drop=True)

df = df.drop(df[df.derived_sex == 'Sex Not Available'].index).reset_index(drop=True)

# Data Preprocessor 

In [ ]:
input_features = [
        'loan_type',
        'loan_purpose',
        'business_or_commercial_purpose',
        'loan_to_value_ratio',
        'interest_rate',
        'hoepa_status',
        'loan_term',
        'property_value',
        'construction_method',
        'occupancy_type',
        'income',
        'debt_to_income_ratio',
        'submission_of_application',
        'aus-1'
]

label_features = ['action_taken']

audit_features = [
    'derived_sex',
    'derived_race',
    'derived_ethnicity'
]

# Bias Measurements

In [ ]:
def approval_rate_by_chosen_field(chosen_field):
    
    field_group = df.groupby(by=chosen_field)

    n = len(field_group)
    print(f"We have {n} groups of values in '{chosen_field}'")

    group_approved_counts = [None] * n
    group_counts = [None] * n
    for i, group_name in enumerate(field_group.groups):
        group_list = df[chosen_field] == group_name
        group_count = len(df[group_list].reset_index(drop=True))
        group_counts[i] = group_count
        group_approved_list = (group_list & approved_list)
        group_approved_count = len(df[group_approved_list].reset_index(drop=True))
        group_approved_counts[i] = group_approved_count
        group_approval_rate = group_approved_count / group_count * 100
        print(f"group_name = {group_name:50} has {group_counts[i]:9} members and {group_approved_counts[i]:9} members were approved for loans. this rate is {group_approval_rate:04.2f}%, which is {group_approval_rate - approval_rate:+05.2f}% the global approval rate")

In [ ]:
approved_list = df.action_taken == 1
approval_count = len(df[approved_list].reset_index(drop=True))
approval_rate = approval_count / len(df) * 100
print(f"the overall approval rate is {approval_rate:05.2f}")

the overall approval rate is 85.23


In [ ]:
approval_rate_by_chosen_field(chosen_field = 'derived_sex')

We have 3 groups of values in 'derived_sex'
group_name = Female                                             has    162703 members and    134832 members were approved for loans. this rate is 82.87%, which is -2.36% the global approval rate
group_name = Joint                                              has    366999 members and    323006 members were approved for loans. this rate is 88.01%, which is +2.78% the global approval rate
group_name = Male                                               has    223808 members and    184395 members were approved for loans. this rate is 82.39%, which is -2.84% the global approval rate


In [ ]:
approval_rate_by_chosen_field(chosen_field = 'derived_race')

We have 7 groups of values in 'derived_race'
group_name = 2 or more minority races                           has      1083 members and       835 members were approved for loans. this rate is 77.10%, which is -8.13% the global approval rate
group_name = American Indian or Alaska Native                   has      5469 members and      4117 members were approved for loans. this rate is 75.28%, which is -9.95% the global approval rate
group_name = Asian                                              has     37689 members and     31361 members were approved for loans. this rate is 83.21%, which is -2.02% the global approval rate
group_name = Black or African American                          has      8552 members and      6686 members were approved for loans. this rate is 78.18%, which is -7.05% the global approval rate
group_name = Joint                                              has     26146 members and     22728 members were approved for loans. this rate is 86.93%, which is +1.70% the g

In [ ]:
approval_rate_by_chosen_field(chosen_field = 'derived_ethnicity')

We have 3 groups of values in 'derived_ethnicity'
group_name = Hispanic or Latino                                 has     36559 members and     28835 members were approved for loans. this rate is 78.87%, which is -6.36% the global approval rate
group_name = Joint                                              has     20711 members and     17829 members were approved for loans. this rate is 86.08%, which is +0.85% the global approval rate
group_name = Not Hispanic or Latino                             has    696240 members and    595569 members were approved for loans. this rate is 85.54%, which is +0.31% the global approval rate
